# Chapter 4 Word2Vec 속도 개선 

앞장과 비교해서 두가지 개선을 추가할 것입니다. 첫 번째 개선으로는 Embedding이라는 새로운 계층을 도입하고 두 번째로는 네거티브 샘플링을 통해 새로운 손실 함수를 도입합니다. 
앞에서 얘기했던 CBOW 모델을 큰 말풍선에 활용을 할 경우에 중간 계산에 많은 시간이 소요됩니다. 원핫 표현과 가중치 행렬의 곱 계산/은닉층과 가중치 행렬의 곱 및 Softmax 계층의 계산이 
시간을 많이 소요하게 만드는 요인입니다. 

Embedding 계층 

자연어 처리 분야에서 단어의 밀집벡터 표현을 단어 임베딩 혹은 단어의 분산 표현이라 합니다. 행렬에서 특정 행을 추출하기란 아주 쉽습니다. 가중치 W가 2차원 넘파이 배열일 때, 
이 가중치로부터 특정 행을 추출하려면 그저 W[2]나 W[5]처럼 원하는 행을 명시하면 끝입니다. 


In [2]:
## 추출 
import numpy as np 
W = np.arange(21).reshape(7, 3)
print(W)
print(W[2])
print(W[5])


[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[6 7 8]
[15 16 17]


In [3]:
## 임베딩 forward method 구현 

class Embedding: 
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
    
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    ## 역전파에서는 반대로 gradient값이다
    ## 안좋은예 
    # def backward(self, dout):
    #     dw, = self.grads
    #     dw[...] = 0
    #     dw[self.idx] = dout
    #     return None 
    
    ## 이 코드는 가중치 기울기 dW를 꺼낸 다음, dw의 원소를 0으로 덮어씁니다. 그리고 앞층에서 전해진 기울기 dout을 idx번째 행에 할당합니다. 
    ## 문제 
    ## idx의 원소가 중복될 때 발생합니다. 
    ## 그래서 할당이 아닌 더하기를 해야합니다. 
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        if GPU:
            np.scatter_add(dW, self.idx, dout)
        else:
            np.add.at(dW, self.idx, dout)
        return None

남은 병목은 은닉층 이후의 처리입니다. 이 병목을 해소하기 위해 네거티브 샘플링이라는 기법을 사용해서 말이죠. 
이 기법은 다중 분류를 이진 부류로 근사하는 것으로 이해하는 게 중점입니다. 

은닉층에서는 특정 단어에 대한 열벡터만을 가지고 와서 계산합니다.

이전까지의 출력층에서는 모든 단어를 대상으로 계산을 수행했습니다. 하지만 여기서는 특정 단어 하나에 주목하여 그 점수만을 계산합니다. 
그리고 시그모이드 함수를 이용해 그 점수를 확률로 변환합니다. 

In [13]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


ModuleNotFoundError: No module named 'ch04'